# HW2

## Exercise 1.1

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
import time

from typing import Final
from sklearn.metrics import mean_squared_error, accuracy_score, classification_report, multilabel_confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier


In [3]:
def OneHotEncoding(df: pd.Series) -> np.ndarray:
    encodedClassNames: list[list[int]] = []
    for i in range(df.shape[0]):
        strClassName = str(df.iloc[i])
        encodedClassName = [
            int(strClassName == 'SEKER'),
            int(strClassName == 'BARBUNYA'),
            int(strClassName == 'BOMBAY'),
            int(strClassName == 'CALI'),
            int(strClassName == 'DERMASON'),
            int(strClassName == 'HOROZ'),
            int(strClassName == 'SIRA')
        ]
        if encodedClassName == [0, 0, 0, 0, 0, 0, 0]:
            raise ValueError(f'Unknown Class Name: {strClassName}')
        else:
            encodedClassNames.append(encodedClassName)

    return np.asarray(encodedClassNames)


In [4]:
# Read in the data
rawData: Final[pd.DataFrame] = pd.read_csv('./Dry_Beans_Dataset.csv')
inputAttributes: Final[pd.DataFrame] = rawData.drop(
    columns=['Class'], inplace=False)

# One hot encoding
encodedClassNames: Final[np.ndarray] = OneHotEncoding(rawData['Class'])

# Normalization
normalizedAttributes: Final[pd.DataFrame] = pd.DataFrame(
    MinMaxScaler()
    .fit(inputAttributes)
    .transform(inputAttributes)
)


## Exercise 1.2

In [66]:
# Setup some constants
DEFAULT_LEARNING_RATE: Final[float] = 0.3
DEFAULT_EPOCHS: Final[int] = 500


In [67]:
def BuildDefaultModel():
    SGD_optimizer: Final = tf.keras.optimizers.SGD(
        learning_rate=DEFAULT_LEARNING_RATE)
    lossFunction = tf.keras.losses.MeanSquaredError()

    model: keras.Sequential = keras.Sequential([
        keras.Input(shape=(16)),
        tf.keras.layers.Dense(
            units=12, activation='sigmoid', name='hidden_layer_1'),
        tf.keras.layers.Dense(
            units=3, activation='sigmoid', name='hidden_layer_2'),
        tf.keras.layers.Dense(units=7, activation='sigmoid', name='output'),
    ], name='Beans_Classifier')

    model.compile(loss=lossFunction, optimizer=SGD_optimizer,
                  metrics=['accuracy'])
    return model


beansClassifier = BuildDefaultModel()


In [62]:
X_Train, X_Test, y_Train, y_Test = train_test_split(
    normalizedAttributes, encodedClassNames, test_size=0.1, random_state=44)


In [69]:
# Compile and Train
startTime = time.time()
beansClassifier.fit(
    x=X_Train, y=y_Train,
    epochs=DEFAULT_EPOCHS,
    validation_data=(X_Test, y_Test), verbose='2')
endTime = time.time()

print(f'Training took {endTime-startTime} seconds')


Epoch 1/500
Epoch 2/500
Epoch 3/500
Epoch 4/500
Epoch 5/500
Epoch 6/500
Epoch 7/500
Epoch 8/500
Epoch 9/500
Epoch 10/500
Epoch 11/500
Epoch 12/500
Epoch 13/500
Epoch 14/500
Epoch 15/500
Epoch 16/500
Epoch 17/500
Epoch 18/500
Epoch 19/500
Epoch 20/500
Epoch 21/500
Epoch 22/500
Epoch 23/500
Epoch 24/500
Epoch 25/500
Epoch 26/500
Epoch 27/500
Epoch 28/500
Epoch 29/500
Epoch 30/500
Epoch 31/500
Epoch 32/500
Epoch 33/500
Epoch 34/500
Epoch 35/500
Epoch 36/500
Epoch 37/500
Epoch 38/500
Epoch 39/500
Epoch 40/500
Epoch 41/500
Epoch 42/500
Epoch 43/500
Epoch 44/500
Epoch 45/500
Epoch 46/500
Epoch 47/500
Epoch 48/500
Epoch 49/500
Epoch 50/500
Epoch 51/500
Epoch 52/500
Epoch 53/500
Epoch 54/500
Epoch 55/500
Epoch 56/500
Epoch 57/500
Epoch 58/500
Epoch 59/500
Epoch 60/500
Epoch 61/500
Epoch 62/500
Epoch 63/500
Epoch 64/500
Epoch 65/500
Epoch 66/500
Epoch 67/500
Epoch 68/500
Epoch 69/500
Epoch 70/500
Epoch 71/500
Epoch 72/500
Epoch 73/500
Epoch 74/500
Epoch 75/500
Epoch 76/500
Epoch 77/500
Epoch 78

In [74]:
# Print the MSE, Accuracy Score, and Confusion Matrices

predicted_y = beansClassifier.predict(X_Test)  # raw continuous outputs
predicted_y_argmaxed = predicted_y.argmax(axis=1)

print(f'MSE of test set is: {mean_squared_error(predicted_y, y_Test)}')
print(f'Accuracy: {accuracy_score(predicted_y_argmaxed, y_Test.argmax(1))}')

print('Precision & Recall:')
print(classification_report(predicted_y_argmaxed, y_Test.argmax(1)))

# This prints an array of 7 matrices, each matrix is 2x2 of [[TT, TF], [FT, FF]]
# Where each index represents the corresponding class name
# in the OneHotEncoding function in the above cell
print('Confusion Matrix:')
print(multilabel_confusion_matrix(
    y_pred=predicted_y_argmaxed, y_true=y_Test.argmax(axis=1)))


MSE of test set is: 0.0232497933910851
Accuracy: 0.9038179148311307
Precision & Recall:
              precision    recall  f1-score   support

           0       0.92      0.98      0.95       199
           1       0.86      0.92      0.89       132
           2       0.98      0.95      0.97        60
           3       0.92      0.92      0.92       164
           4       0.94      0.88      0.91       383
           5       0.93      0.97      0.95       175
           6       0.82      0.80      0.81       249

    accuracy                           0.90      1362
   macro avg       0.91      0.92      0.91      1362
weighted avg       0.90      0.90      0.90      1362

Confusion Matrix:
[[[1146    3]
  [  17  196]]

 [[1210   11]
  [  20  121]]

 [[1301    3]
  [   1   57]]

 [[1185   13]
  [  13  151]]

 [[ 956   45]
  [  23  338]]

 [[1175    6]
  [  12  169]]

 [[1068   50]
  [  45  199]]]


## Exercise 2

In [75]:
def generateValidationData(inputDF: pd.DataFrame, expectedOutputs: np.ndarray,
                           trainIndexes: np.ndarray,
                           testIndexes: np.ndarray) -> tuple[pd.Series, pd.Series, np.ndarray, np.ndarray]:
    X_Train = inputDF.iloc[trainIndexes]
    X_Test = inputDF.iloc[testIndexes]

    y_Train = expectedOutputs[trainIndexes]
    y_Test = expectedOutputs[testIndexes]

    return X_Train, X_Test, y_Train, y_Test


In [77]:
# Do 10-fold validation

KFolder = KFold(n_splits=10)
mseScores: list[float] = []
accuracyScores: list[float] = []

for trainIndexes, testIndexes in KFolder.split(normalizedAttributes):
    X_Train, X_Test, y_Train, y_Test = generateValidationData(
        normalizedAttributes, encodedClassNames, trainIndexes, testIndexes)

    beansClassifier.fit(
        x=X_Train, y=y_Train,
        epochs=DEFAULT_EPOCHS,
        validation_data=(X_Test, y_Test))

    predicted_y: np.ndarray = beansClassifier.predict(X_Test)

    mseScores.append(mean_squared_error(y_Test, predicted_y))
    accuracyScores.append(accuracy_score(
        predicted_y.argmax(axis=1), y_Test.argmax(axis=1)))


Epoch 1/500
383/383 [==============================] - 0s 1ms/step - loss: 0.0214 - accuracy: 0.9134 - val_loss: 0.0193 - val_accuracy: 0.9236
Epoch 2/500
383/383 [==============================] - 0s 949us/step - loss: 0.0213 - accuracy: 0.9131 - val_loss: 0.0191 - val_accuracy: 0.9236
Epoch 3/500
383/383 [==============================] - 0s 963us/step - loss: 0.0213 - accuracy: 0.9139 - val_loss: 0.0195 - val_accuracy: 0.9244
Epoch 4/500
383/383 [==============================] - 0s 953us/step - loss: 0.0213 - accuracy: 0.9134 - val_loss: 0.0195 - val_accuracy: 0.9222
Epoch 5/500
383/383 [==============================] - 0s 962us/step - loss: 0.0213 - accuracy: 0.9135 - val_loss: 0.0196 - val_accuracy: 0.9244
Epoch 6/500
383/383 [==============================] - 0s 971us/step - loss: 0.0212 - accuracy: 0.9140 - val_loss: 0.0190 - val_accuracy: 0.9244
Epoch 7/500
383/383 [==============================] - 0s 991us/step - loss: 0.0212 - accuracy: 0.9146 - val_loss: 0.0194 - val_accu

In [78]:
print(
    f'Accuracy Scores: {accuracyScores}\nAverage accuracy is: {np.average(accuracyScores)}\n')
print(f'MSE Loss: {mseScores}\nAverage MSE is: {np.average(mseScores)}')


Accuracy Scores: [0.9287812041116006, 0.919911829537105, 0.9213813372520205, 0.9294636296840558, 0.9287288758265981, 0.9287288758265981, 0.9309331373989713, 0.9353416605437178, 0.9184423218221895, 0.9155033063923586]
Average accuracy is: 0.9257216178395214

MSE Loss: [0.01714564975196725, 0.01841759192824956, 0.018134206557099655, 0.01678210401819093, 0.01679172421283575, 0.01622916924843049, 0.014924349812376368, 0.014846849830030638, 0.01745446732690939, 0.018019976349228965]
Average MSE is: 0.0168746089035319


## Exercise 3

In [35]:
def BuildModel(numNodesLayer1=12, numNodesLayer2=3, learningRate=DEFAULT_LEARNING_RATE):
    SGD_optimizer: Final = tf.keras.optimizers.SGD(learning_rate=learningRate)
    lossFunction = tf.keras.losses.MeanSquaredError()

    model: keras.Sequential = keras.Sequential([
        tf.keras.layers.Dense(
            input_dim=16,
            units=numNodesLayer1, activation='sigmoid', name='hidden_layer_1'),
        tf.keras.layers.Dense(
            units=numNodesLayer2, activation='sigmoid', name='hidden_layer_2'),
        tf.keras.layers.Dense(units=7, activation='sigmoid', name='output'),
    ], name='Beans_Classifier')

    model.compile(loss=lossFunction, optimizer=SGD_optimizer,
                  metrics=['accuracy'])
    return model

wrappedBeansClassifier = KerasClassifier(build_fn=BuildModel)

# parameters passed to BuildModel(...)
param_grid = dict(
    nb_epoch=np.array([400, 500, 600]),
    learningRate=np.array([0.1, 0.3, 0.6]),
    numNodesLayer1=np.array([12, 13, 14]),
    numNodesLayer2=np.array([3, 4, 5]),
)

grid = GridSearchCV(estimator=wrappedBeansClassifier,
                    param_grid=param_grid, n_jobs=-1, cv=10)

grid_result = grid.fit(X_Train, y_Train)


/var/folders/n9/fnc__vj96pbc8hshz5gbdl7r0000gn/T/ipykernel_18524/1612889540.py:22: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  wrappedBeansClassifier = KerasClassifier(build_fn=BuildModel2)


{'build_fn': <function BuildModel2 at 0x13ef96ee0>}
{'cv': 10, 'error_score': nan, 'estimator__build_fn': <function BuildModel2 at 0x13ef96ee0>, 'estimator': <keras.wrappers.scikit_learn.KerasClassifier object at 0x13f0467f0>, 'n_jobs': -1, 'param_grid': {'nb_epoch': array([400, 500, 600]), 'learningRate': array([0.1, 0.3, 0.6]), 'numNodesLayer1': array([12, 13, 14]), 'numNodesLayer2': array([3, 4, 5])}, 'pre_dispatch': '2*n_jobs', 'refit': True, 'return_train_score': False, 'scoring': None, 'verbose': 0}


2022-05-03 20:41:53.807997: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-03 20:41:53.808051: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-03 20:41:53.808055: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the ap

12 3 0.1
12 3 0.1
12 3 0.1
12 3 0.1
12 3 0.1
12 3 0.1
12 4 0.1
12 12 3 0.1
12 3 0.1
3 0.1
12 4 0.1
12 3 0.1
39/39 [==============================] - 0s 1ms/step - loss: 0.1366 - accuracy: 0.2718
12 4 0.1
12 4 0.1
12 4 0.1
12 4 0.1
12 4 0.1
12 4 0.1
12 5 0.1
12 4 0.1
12 4 0.1
12 5 0.1
12 5 0.1
12 5 0.1
39/39 [==============================] - 0s 2ms/step - loss: 0.1289 - accuracy: 0.1698
12 12 5 5 0.1
0.1
12 5 0.1
12 5 0.1
12 5 0.1
 1/39 [..............................] - ETA: 4s - loss: 0.1250 - accuracy: 0.343812 5 0.1
13 3 0.1
13 3 0.1
13 3 0.1
13 3 0.1
39/39 [==============================] - 0s 4ms/step - loss: 0.1314 - accuracy: 0.2571
13 3 0.1
39/39 [==============================] - 0s 2ms/step - loss: 0.1314 - accuracy: 0.2629
13 3 0.1
39/39 [==============================] - 0s 1ms/step - loss: 0.1321 - accuracy: 0.2718
13 3 0.1
39/39 [==============================] - 0s 2ms/step - loss: 0.1341 - accuracy: 0.2571
13 4 0.1
13 4 0.1
13 4 0.1
13 4 0.1
13 4 0.1
13 4 0.1
39/39 [==

In [79]:
# Big reveal of best parameters
grid_result.best_params_

{'learningRate': 0.1,
 'nb_epoch': 500,
 'numNodesLayer1': 12,
 'numNodesLayer2': 4}